# LDA topic modeling with a curated dataset

This example uses the [`gensim`](https://radimrehurek.com/gensim/index.html) library for building the topic model.

Depending on your collection size, this example will take between 10 minutes to an hour+ to run. 

In [39]:
import os

import warnings
warnings.filterwarnings('ignore')

In [40]:
import gensim
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

import pyLDAvis.gensim

pyLDAvis.enable_notebook()

stop_words = set(stopwords.words('english'))

Initialize a dataset object. 

In [41]:
from tdm_client import Dataset

dset = Dataset('a517ef1f-0794-48e4-bea1-ac4fb8b312b4')

In [42]:
len(dset)

1000

In [43]:
dset.query()

'q=shakespeare&start=0&rows=20&fq=yearPublished%3A%5B1900%20TO%202019%5D&fq=category%3A(%22Literature%20(General)%22%20OR%20%22English%20literature%22)'

Define a function for processing tokens from the extracted features for volumes in the curated dataset. This function:

* lowercases all tokens
* discards all tokens less than 4 characters
* discards non alphabetical tokens - e.g. --9
* removes stopwords using NLTK's stopword list
* Lemmatizes the token using NLTK's [WordNetLemmatizer](https://www.nltk.org/_modules/nltk/stem/wordnet.html)

In [44]:
def process_token(token):
    token = token.lower()
    if len(token) < 4:
        return
    if not(token.isalpha()):
        return
    if token in stop_words:
        return
    return WordNetLemmatizer().lemmatize(token)

Loop through the volumes in the dataset and make a list of tokens for each volume and then add to a list of the 25 documents in the dataset. We are limiting this example to 25 documents to limit the time it takes to run during demonstrations.

In [45]:
documents = []

for doc_n, volume in enumerate(dset.get_features()):
    this_doc = []
    try:
        pages = volume['features']['pages']
    except:
        continue
    for pn, page in enumerate(pages):
        body = page.get('body')
        if body is not None:
            for token, pos_count in body.get('tokenPosCount', {}).items():
                t = process_token(token)
                if t is None:
                    continue
                for pos, n in pos_count.items():
                    this_doc += [t] * n
    documents.append(this_doc)
    if doc_n >= 25:
        break
                

Build a gensim dictionary and corpus and then train the model.

In [46]:
dictionary = gensim.corpora.Dictionary(documents)

In [47]:
dictionary.filter_extremes(no_below=len(documents) * .10, no_above=0.5)

bow_corpus = [dictionary.doc2bow(doc) for doc in documents]


# train model, this might take some time
model = gensim.models.LdaModel(
    corpus=bow_corpus,
    id2word=dictionary,
    num_topics=5,
    passes=15
)


Visualize the model using [`pyLDAvis`](https://pyldavis.readthedocs.io/en/latest/).

In [48]:
pyLDAvis.gensim.prepare(model, bow_corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.087444 -0.002296       1        1  47.927330
1      0.028763 -0.007707       2        1  20.134264
4     -0.174840  0.029102       3        1  13.552016
3      0.004942 -0.134078       4        1  12.295904
0      0.053691  0.114979       5        1   6.090487, topic_info=      Category        Freq           Term       Total  loglift  logprob
7933   Default  189.000000           arab  189.000000  30.0000  30.0000
3699   Default  129.000000         hawkes  129.000000  29.0000  29.0000
4634   Default  158.000000          wilde  158.000000  28.0000  28.0000
3698   Default  144.000000       hathaway  144.000000  27.0000  27.0000
281    Default  254.000000    bolingbroke  254.000000  26.0000  26.0000
1520   Default  126.000000           joan  126.000000  25.0000  25.0000
7750   Default  129.000000        siddons  129.000000  24.0000  24.0000
1604   Default  485.000000       lighting  485.000000  23.0000  23.0000
7511   Default  126.000000         kemble  126.000000  22.0000  22.0000
3827   Default   98.000000         jesuit   98.000000  21.0000  21.0000
985    Default  105.000000        epitaph  105.000000  20.0000  20.0000
6354   Default  101.000000           eden  101.000000  19.0000  19.0000
8141   Default  105.000000          egypt  105.000000  18.0000  18.0000
6357   Default  100.000000       egyptian  100.000000  17.0000  17.0000
4874   Default   88.000000  commemoration   88.000000  16.0000  16.0000
4702   Default   96.000000    apocalyptic   96.000000  15.0000  15.0000
5720   Default  397.000000      repertory  397.000000  14.0000  14.0000
7827   Default   96.000000     translator   96.000000  13.0000  13.0000
5737   Default   73.000000          revel   73.000000  12.0000  12.0000
4862   Default  104.000000      coleridge  104.000000  11.0000  11.0000
5897   Default   68.000000   tercentenary   68.000000  10.0000  10.0000
4494   Default   61.000000        terence   61.000000   9.0000   9.0000
3086   Default   74.000000     apocalypse   74.000000   8.0000   8.0000
9413   Default   86.000000         faucit   86.000000   7.0000   7.0000
2127   Default   86.000000     promptbook   86.000000   6.0000   6.0000
1268   Default  105.000000         harris  105.000000   5.0000   5.0000
4062   Default  109.000000         online  109.000000   4.0000   4.0000
4714   Default  152.000000          ariel  152.000000   3.0000   3.0000
6231   Default   98.000000     compositor   98.000000   2.0000   2.0000
2015   Default  102.000000       platform  102.000000   1.0000   1.0000
...        ...         ...            ...         ...      ...      ...
12307   Topic5    4.816847          rouen    7.435257   2.3643  -7.5326
12331   Topic5    4.816388        watkins    7.436657   2.3640  -7.5327
9928    Topic5    4.820095          trite    7.492385   2.3573  -7.5319
12223   Topic5    4.817853     corpulence    7.496889   2.3563  -7.5324
12324   Topic5    4.816599           tile    7.497293   2.3560  -7.5327
11353   Topic5    6.372090        hofstra    9.969677   2.3508  -7.2528
7657    Topic5    7.150743         prouty   11.386489   2.3332  -7.1375
7365    Topic5   11.822168   evolutionary   19.252834   2.3108  -6.6347
4961    Topic5    4.039597         deluge    6.602806   2.3071  -7.7086
9867    Topic5    4.039580        plenary    6.618988   2.3046  -7.7086
5781    Topic5   15.693852       schiller   26.630135   2.2697  -6.3515
2451    Topic5    7.928204      shorthand   13.285213   2.2822  -7.0343
2415    Topic5   11.810246      semiotics   20.149563   2.2642  -6.6358
4494    Topic5   32.817020        terence   61.131805   2.1764  -5.6138
1306    Topic5   16.493687    historicism   29.277042   2.2246  -6.3017
1520    Topic5   60.823093           joan  126.038773   2.0698  -4.9968
5525    Topic5   15.695918        orleans   27.864689   2.2245  -6.3513
8973    Topic5   10.254889       humanism   17.4604